In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# sklearn proprocessing
from sklearn.mixture import GaussianMixture as GMM

#sklearn model
from sklearn.ensemble import RandomForestClassifier

#sklearn model selection
from sklearn.model_selection import GridSearchCV, cross_val_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv('../input/train.csv', header=None)
train_labels = pd.read_csv('../input/trainLabels.csv', header=None)
test_data = pd.read_csv('../input/test.csv', header=None)
X_train = np.asarray(train_data)
Y_train = np.asarray(train_labels).ravel()
X_test= np.asarray(test_data)

In [ ]:
# use Gaussian Mixture model to cover the distrubution of the data
X_all = np.r_[X_train, X_test]
lowest_aic = np.inf
best_gmm = None
aics = []

n_components_range = range(1,7)
cv_types = ['spherical', 'tied', 'diag', 'full']
for cv_type in cv_types:
    for n_components in n_components_range:
        gmm = GMM(n_components=n_components, covariance_type=cv_type, reg_covar=0.0001)
        gmm.fit(X_all)
        aics.append(gmm.aic(X_all))
        if(aics[-1] < lowest_aic):
            lowest_aic = aics[-1]
            best_gmm = gmm

            

In [ ]:
# use mixture model to transform data from 40 dims -> best model's dims
g = best_gmm
print('dimension of data after transformed by best model: ', g.n_components)
g.fit(X_all)

In [ ]:
X = g.predict_proba(X_train)

In [ ]:
# random forest + grid search cv
clf=RandomForestClassifier(n_estimators=1000, criterion='entropy', max_depth=5, min_samples_split=1.0,
  min_samples_leaf=3, max_features='auto', bootstrap=False, oob_score=False, n_jobs=-1, random_state=33,
  verbose=0)


In [ ]:
 grid_search = GridSearchCV(clf, param_grid=dict(), verbose=3,scoring='accuracy',cv=5).fit(X, Y_train)
 ###print result
 print(grid_search.best_estimator_)
 #report(grid_search.grid_scores_)


In [ ]:
###predict best fit
svc=grid_search.best_estimator_.fit(X,Y_train)

print ('best'), grid_search.best_estimator_.score(X, Y_train)
scores = cross_val_score(svc, X, Y_train,cv=5,scoring='accuracy')
print (scores.mean(),scores.min())
print (scores)

In [ ]:
X_t =g.predict_proba(X_test)

y_pred=grid_search.best_estimator_.predict(X_t)
print (y_pred.shape)
print (g)


y_pred=grid_search.best_estimator_.predict(X_t)
print (y_pred.shape)
print (g)
 
# submit data
submission = pd.DataFrame({'Id':np.arange(1,len(y_pred)+1), 'Solution':y_pred})
submission.to_csv(index=None, path_or_buf='svm_submission.csv')
